In [1]:

from IPython.display import display, HTML
import ipywidgets as widgets
import pandas as pd

# Load dataset
filepath = '/content/movies.csv'
movies_data = pd.read_csv(filepath, encoding='latin-1')
movies_data.columns = movies_data.columns.str.strip().str.lower()
movies_data['imdb ratings'] = pd.to_numeric(movies_data['imdb ratings'], errors='coerce')
movies_data['year'] = movies_data['year'].astype(str)

def extract_youtube_id(trailer):
    if pd.isna(trailer):
        return ""
    if "watch?v=" in trailer:
        return trailer.split("watch?v=")[-1].split("&")[0]
    elif "youtu.be/" in trailer:
        return trailer.split("youtu.be/")[-1].split("?")[0]
    return trailer.strip()

# Widgets
search_bar = widgets.Text(placeholder="🔍 Search for movies...", layout=widgets.Layout(width='45%'))
search_filter = widgets.Dropdown(
    options=['movie title', 'genres', 'actors', 'director', 'music composer', 'singers', 'songs', 'year', 'platform', 'age group'],
    value='movie title',
    description='Filter:',
    layout=widgets.Layout(width='30%')
)
search_button = widgets.Button(description="Search", button_style='primary', icon='search', layout=widgets.Layout(width='20%'))
output_area = widgets.Output()
suggestions_area = widgets.Output()

# Display results
def display_results(results):
    with output_area:
        output_area.clear_output()
        if results.empty:
            display(HTML("<h3 style='color:white;'>No results found!</h3>"))
            return

        html = """
        <style>
        body { background-color: black; color: white; font-family: Arial, sans-serif; }
        .movie-card {
            display: flex;
            flex-wrap: wrap;
            margin: 30px 0;
            padding: 20px;
            border: 1px solid #444;
            border-radius: 12px;
            background-color: #1a1a1a;
            transition: all 0.3s ease-in-out;
        }
        .trailer-container {
            position: relative;
            width: 250px;
            height: 350px;
            margin-right: 20px;
            border-radius: 12px;
            overflow: hidden;
            transition: all 0.5s ease;
            cursor: pointer;
        }
        .poster, .iframe-hover {
            width: 100%;
            height: 100%;
            object-fit: cover;
            position: absolute;
            top: 0;
            left: 0;
            transition: opacity 0.4s ease;
            border-radius: 12px;
        }
        .iframe-hover {
            opacity: 0;
            pointer-events: none;
            border: none;
        }
        .trailer-container:hover .poster {
            opacity: 0;
        }
        .trailer-container:hover .iframe-hover {
            opacity: 1;
            pointer-events: auto;
        }
        .movie-details {
            max-width: 600px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
            color: white;
            transition: opacity 0.3s ease;
        }
        .movie-details h3 { font-size: 22px; margin-bottom: 10px; color: #00aced; }
        .movie-details p { font-size: 14px; margin: 4px 0; }
        .movie-details p strong { color: #ccc; }

        .toggle-button {
            background-color: #444;
            color: white;
            padding: 6px 14px;
            border-radius: 6px;
            margin-top: 10px;
            cursor: pointer;
            display: inline-block;
            font-size: 13px;
            border: none;
        }

        .expanded .trailer-container {
            width: 100% !important;
            height: 500px !important;
        }
        .expanded .movie-details {
            display: none !important;
        }
        .back-button {
            background-color: #e74c3c;
            color: white;
            padding: 6px 14px;
            border-radius: 6px;
            margin-top: 10px;
            cursor: pointer;
            display: none;
            font-size: 13px;
            border: none;
        }
        .expanded .back-button {
            display: inline-block;
        }
        </style>

        <script>
        function toggleTrailer(id) {
            const card = document.getElementById('movie-card-' + id);
            const trailer = card.querySelector('.iframe-hover');
            const toggleBtn = card.querySelector('.toggle-button');
            const backBtn = card.querySelector('.back-button');

            if (card.classList.contains('expanded')) {
                card.classList.remove('expanded');
                trailer.src = trailer.src.replace("autoplay=1", "autoplay=0");
                toggleBtn.style.display = 'inline-block';
                backBtn.style.display = 'none';
            } else {
                card.classList.add('expanded');
                trailer.src = trailer.src.replace("autoplay=0", "autoplay=1");
                toggleBtn.style.display = 'none';  // Hide the Max View button
                backBtn.style.display = 'inline-block';  // Show the Back to Details button
            }
        }

        function backButtonClick(id) {
            const card = document.getElementById('movie-card-' + id);
            const trailer = card.querySelector('.iframe-hover');
            const toggleBtn = card.querySelector('.toggle-button');
            const backBtn = card.querySelector('.back-button');

            card.classList.remove('expanded');
            trailer.src = trailer.src.replace("autoplay=1", "autoplay=0");
            toggleBtn.style.display = 'inline-block';  // Show the Max View button
            backBtn.style.display = 'none';  // Hide the Back to Details button
        }
        </script>
        """

        for idx, row in results.iterrows():
            poster = row['poster_url'] if pd.notna(row['poster_url']) else "https://via.placeholder.com/200x300?text=No+Image"
            title = row['movie title']
            video_id = extract_youtube_id(row.get('trailer'))
            genres = row['genres'] if pd.notna(row['genres']) else "N/A"
            imdb = row['imdb ratings'] if pd.notna(row['imdb ratings']) else "N/A"
            actors = row['actors'] if pd.notna(row['actors']) else "N/A"
            director = row['director'] if pd.notna(row['director']) else "N/A"
            composer = row['music composer'] if pd.notna(row['music composer']) else "N/A"
            singers = row['singers'] if pd.notna(row['singers']) else "N/A"
            songs = row['songs'] if pd.notna(row['songs']) else "N/A"
            budget = row['budget (in cr)'] if pd.notna(row['budget (in cr)']) else "N/A"
            box_office = row['box office collection (in cr)'] if pd.notna(row['box office collection (in cr)']) else "N/A"
            year = row['year'] if pd.notna(row['year']) else "N/A"
            platform = row['platform'] if pd.notna(row['platform']) else "N/A"
            age_group = row['age group'] if pd.notna(row['age group']) else "N/A"

            iframe = f"""
            <iframe class='iframe-hover'
                src='https://www.youtube.com/embed/{video_id}?autoplay=0&mute=1&controls=1'
                allow='autoplay; encrypted-media' allowfullscreen></iframe>""" if video_id else ""

            html += f"""
            <div class="movie-card" id="movie-card-{video_id}">
                <div class="trailer-container" onclick="toggleTrailer('{video_id}')">
                    <img class="poster" src="{poster}" alt="{title}">
                    {iframe}
                </div>
                <div class="movie-details">
                    <h3>{title} ({year})</h3>
                    <p><strong>Genres:</strong> {genres}</p>
                    <p><strong>IMDb Ratings:</strong> {imdb}/10</p>
                    <p><strong>Actors:</strong> {actors}</p>
                    <p><strong>Director:</strong> {director}</p>
                    <p><strong>Music Composer:</strong> {composer}</p>
                    <p><strong>Singers:</strong> {singers}</p>
                    <p><strong>Songs:</strong> {songs}</p>
                    <p><strong>Budget:</strong> ₹{budget} Cr</p>
                    <p><strong>Box Office:</strong> ₹{box_office} Cr</p>
                    <p><strong>Platform:</strong> {platform}</p>
                    <p><strong>Age Group:</strong> {age_group}</p>
                    <button class="toggle-button" onclick="toggleTrailer('{video_id}')">▶ Max View</button>
                    <button class="back-button" onclick="backButtonClick('{video_id}')">⬅ Back to Details</button>
                </div>
            </div>
            """

        display(HTML(html))

# Search logic
def search_movies(_=None):
    query = search_bar.value.strip()
    filter_col = search_filter.value
    if not query:
        output_area.clear_output()
        display(HTML("<h3 style='color:white;'>Please enter a search term.</h3>"))
        return
    results = movies_data[movies_data[filter_col].astype(str).str.contains(query, case=False, na=False)]
    results = results.sort_values(by='imdb ratings', ascending=False)
    display_results(results)

# Suggestion feature
def update_suggestions(change):
    query = search_bar.value.strip()
    filter_by = search_filter.value
    with suggestions_area:
        suggestions_area.clear_output()
        if query:
            column_data = movies_data[filter_by].dropna()
            column_data = column_data.apply(lambda x: x.split(',') if ',' in x else [x]).explode().str.strip()
            filtered = column_data[column_data.str.contains(query, case=False)].unique()
            if filtered.any():
                buttons = []
                for s in filtered[:10]:
                    b = widgets.Button(description=s, layout=widgets.Layout(width='auto'))
                    b.on_click(lambda btn, s=s: on_suggestion_click(s))
                    buttons.append(b)
                display(widgets.VBox(buttons))

def on_suggestion_click(suggestion):
    search_bar.value = suggestion
    search_movies()

# Connect widgets
search_bar.observe(update_suggestions, names='value')
search_button.on_click(search_movies)

# Display UI
ui = widgets.VBox([widgets.HTML("<h2 style='color:white;'>🎬 Movie Recommendation System </h2>"),
                   widgets.HBox([search_bar, search_filter, search_button]),
                   suggestions_area,
                   output_area])
display(ui)


